In [ ]:
# 安装conda环境
!pip install -q condacolab
import condacolab
condacolab.install()

# 创建并激活新的conda环境
!conda create -n minigptv python=3.9 -y
!conda activate minigptv

# 安装cudatoolkit
!conda install -c anaconda cudatoolkit -y

# 使用pip安装其他依赖项
!pip install torch==2.0.0 torchaudio torchvision huggingface-hub==0.18.0 matplotlib==3.7.0 psutil==5.9.4 iopath pyyaml==6.0 regex==2022.10.31 tokenizers==0.13.2 tqdm==4.64.1 transformers==4.30.0 timm==0.6.13 webdataset==0.2.48 omegaconf==2.3.0 opencv-python==4.7.0.72 decord==0.6.0 peft==0.2.0 sentence-transformers gradio==3.47.1 accelerate==0.20.3 bitsandbytes==0.37.0 scikit-image visual-genome wandb ipykernel


In [ ]:
# 克隆仓库并切换目录
!git clone https://github.com/Czi24/MiniGPT-4-20240710.git

# 使用 gdown 下载文件
# https://drive.google.com/file/d/1RY9jV0dyqLX-o38LrumkKRh6Jtaop58R/view?usp=sharing

import gdown

# 使用 gdown 下载文件
gdown.download("https://drive.google.com/uc?id=1RY9jV0dyqLX-o38LrumkKRh6Jtaop58R", output=None, quiet=False)

# 下载模型文件
# 方式一
from huggingface_hub import snapshot_download
snapshot_download(repo_id="Vision-CAIR/vicuna-7b", local_dir="vicuna-7b")

# 方式二
# 克隆模型存储库到指定路径
# !git clone https://huggingface.co/Vision-CAIR/vicuna-7b

Cloning into 'MiniGPT-4-20240710'...
remote: Enumerating objects: 1797, done.
remote: Counting objects: 100% (884/884), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 1797 (delta 719), reused 658 (delta 658), pack-reused 913
Receiving objects: 100% (1797/1797), 65.21 MiB | 16.52 MiB/s, done.
Resolving deltas: 100% (1047/1047), done.


Downloading...
From (original): https://drive.google.com/uc?id=1RY9jV0dyqLX-o38LrumkKRh6Jtaop58R
From (redirected): https://drive.google.com/uc?id=1RY9jV0dyqLX-o38LrumkKRh6Jtaop58R&confirm=t&uuid=02935ff6-8097-4485-a5ee-9eb226e12f77
To: /content/prerained_minigpt4_7b.pth
100%|██████████| 37.9M/37.9M [00:02<00:00, 16.6MB/s]


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

'/content/vicuna-7b'

In [ ]:
# 切换到克隆的仓库目录
%cd MiniGPT-4-20240710
%ls

# 查看当前工作目录
!pwd
# %cd ..

/content/MiniGPT-4-20240710
CODE_OF_CONDUCT.md  environment.yml  examples_v2/      minigpt4/           prompts/        train.py
dataset/            eval_configs/    figs/             MiniGPT4_Train.md   README.md
demo.py             eval_scripts/    LICENSE_Lavis.md  MiniGPTv2.pdf       SECURITY.md
demo_v2.py          examples/        LICENSE.md        MiniGPTv2_Train.md  train_configs/
/content/MiniGPT-4-20240710


In [ ]:
# 指定配置文件路径
config_file_path = 'minigpt4/configs/models/minigpt4_vicuna0.yaml'
new_llama_model_path = '/content/vicuna-7b'

# 读取现有的配置文件
with open(config_file_path, 'r') as file:
    lines = file.readlines()

# 修改 llama_model 路径
with open(config_file_path, 'w') as file:
    for line in lines:
        if line.strip().startswith('llama_model:'):
            file.write(f'  llama_model: "{new_llama_model_path}"\n')
        else:
            file.write(line)

# 验证修改
with open(config_file_path, 'r') as file:
    updated_config = file.read()
    print(updated_config)


model:
  arch: minigpt4

  # vit encoder
  image_size: 224
  drop_path_rate: 0
  use_grad_checkpoint: False
  vit_precision: "fp16"
  freeze_vit: True
  freeze_qformer: True

  # Q-Former
  num_query_token: 32

  # generation configs
  prompt: ""

  llama_model: "/content/vicuna-7b"

preprocess:
    vis_processor:
        train:
          name: "blip2_image_train"
          image_size: 224
        eval:
          name: "blip2_image_eval"
          image_size: 224
    text_processor:
        train:
          name: "blip_caption"
        eval:
          name: "blip_caption"



In [ ]:
# 指定配置文件路径
config_file_path = 'eval_configs/minigpt4_eval.yaml'
new_llama_model_path = '/content/prerained_minigpt4_7b.pth'

# 读取现有的配置文件
with open(config_file_path, 'r') as file:
    lines = file.readlines()

# 修改 llama_model 路径
with open(config_file_path, 'w') as file:
    for line in lines:
        if line.strip().startswith('ckpt:'):
            file.write(f"  ckpt: '{new_llama_model_path}'\n")
        else:
            file.write(line)

# 验证修改
with open(config_file_path, 'r') as file:
    updated_config = file.read()
    print(updated_config)


model:
  arch: minigpt4
  model_type: pretrain_vicuna0
  max_txt_len: 160
  end_sym: "###"
  low_resource: True
  prompt_template: '###Human: {} ###Assistant: '
  ckpt: '/content/prerained_minigpt4_7b.pth'


datasets:
  cc_sbu_align:
    vis_processor:
      train:
        name: "blip2_image_eval"
        image_size: 224
    text_processor:
      train:
        name: "blip_caption"

run:
  task: image_text_pretrain



In [ ]:
!python3 demo.py --cfg-path eval_configs/minigpt4_eval.yaml --gpu-id 0


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
Initializing Chat
Loading checkpoint shards: 100% 2/2 [00:13<00:00,  6.90s/it]
Loading Q-Former
Loading Q-Former Done
Load MiniGPT-4 Checkpoint: /content/prerained_minigpt4_7b.pth
Initialization Finished
/content/MiniGPT-4-20240710/demo.py:171: GradioDeprecationWarning: The `enable_queue` parameter has been deprecated. Please use the `.queue()` method instead.
  demo.launch(share=True, enable_queue=True)
Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.47.1, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://f8eec29422096d59e3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spac